In [85]:
%run ./FUNCIONES.ipynb

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import tabula
from tabula import read_pdf
import numpy as np
import pandas as pd
import re as regex
df = read_pdf('1088878.pdf', guess=False, stream=True, pages='all', multiple_tables=True,  pandas_options={'header':None})

In [4]:
pd.set_option('display.max_colwidth', -1)  #Veo hasta la última palabra
pd.set_option('display.max_rows', 1000)

In [6]:
df[0] = df[0].drop(df[0].head(12).index, axis=0)
df[1:] = [x.drop(x.head(5).index, axis=0) for x in df[1:]]

In [9]:
df = [x.drop(x.tail(2).index, axis=0) for x in df]

In [12]:
df[6] = df[6].drop(df[6].tail(13).index, axis=0)

In [16]:
df[1].loc[33, 1] = 'Review and analyze and annotate trial exhibits'
df[1].loc[33, 0] = 'RMW'

In [22]:
df[5].loc[32, 1] = 'Review and analyze client depositions and expert deposition'
df[5].loc[34, 1] = 'Draft outline of opening statement and closing argument'
df[5].loc[32, 0] = '9/17/2017 RMW'
df[5].loc[34, 0] = '9/18/2017 RMW'

In [70]:
for i in range(len(df)):
    print(i)
    df[i].head(5)

0


,0,1
12,5/4/2017 RMW Receive and review RO Summary,0.30 150.00
13,NaN,500.00/hr
14,RMW Review and analyze case files for developing trial strategy,"2.00 1,000.00"
15,NaN,500.00/hr
16,5/23/2017 EJ Draft email to Lisa Tudzin (NP) regarding available dates for deposition,0.10 NO CHARGE


1


,0,1,2
5,5/30/2017 ARS,"Draft, review and revise email to client",0.20 65.00
6,NaN,NaN,325.00/hr
7,ARS,Phone call to client,0.40 130.00
8,NaN,NaN,325.00/hr
9,5/31/2017 RE,Prepare documents for Donna Howton's Deposition,0.20 35.00


2


...

,0
5,7/27/2017 ARS Review email from opposing counsel Tudzin regarding dealership 0.10 NO CHARGE
6,deposition schedule 325.00/hr
7,"ARS Draft, review and revise email to opposing counsel Tudzin regarding 0.10 NO CHARGE"
8,dealership deposition schedule 325.00/hr
9,"8/3/2017 RE Draft, review and revise Notice of Change of Address 0.10 NO CHARGE"


5


,0,1,2
5,NaN,NaN,Hrs/Rate Amount
6,9/12/2017 JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 2",0.50 125.00
7,NaN,NaN,250.00/hr
8,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 3",0.50 125.00
9,NaN,NaN,250.00/hr


6


,0
5,9/19/2017 JRU Prepare timeline chart 2.50 625.00
6,250.00/hr
7,"JRU Draft, review and revise joint witness list 0.70 175.00"
8,250.00/hr
9,RE Prepare and serve Civil Subpoenas and On Call Agreements for 0.60 105.00


# df0

In [71]:
rows_per_row = [1,1,2,2,2,2,2,1,1,2,2]
Date0, Name0, Descripcion0 = Date_Name_Descripcion(df[0].loc[:, 0], rows_per_row)
Hours0, Rates0, Totals0 = Hours_Rates_Totals(df[0].loc[:, 1])

In [72]:
df0 = pd.concat([Date0.rename('Date'), Name0.rename('Names'), Descripcion0.rename('Description'),
                     Hours0.rename('Hours'), Rates0.rename('Rates'), Totals0.rename('Totals')], axis=1)
df0 = df0.dropna().reset_index(drop=True)
df0

,Date,Names,Description,Hours,Rates,Totals
0,5/4/2017,RMW,Receive and review RO Summary,0.30,500.0,150.0
1,5/4/2017,RMW,Review and analyze case files for developing trial strategy,2.00,500.0,1000.0
2,5/24/2017,EJ,"Draft, prepare, and serve Plaintiff's Amended Notice of Deposition of Rancho Chrysler (6-5-17)",0.80,175.0,140.0
3,5/25/2017,ARS,"Draft, review and revise email to Donna Howton",0.10,325.0,32.5
4,5/26/2017,RE,"Draft, review and revise notice in lieu of and notice regarding financial information",0.50,175.0,87.5
5,5/30/2017,EJ,"Draft, prepare, and serve Plaintiff's Objections to Notice of Deposition of Mrs. Howton (6-1-17)",0.20,175.0,35.0


# df1

In [88]:
rows_per_row = [1,1,1,2,1,1,1,2,1,1,1,1,2,1,1,1]
Date1, Name1 = Date_Name(df[1].loc[:, 0])
Descripcion1 = Descripcion_func(df[1].loc[:, 1], rows_per_row)
Hours1, Rates1, Totals1 = Hours_Rates_Totals(df[1].loc[:, 2])

In [89]:
df1 = LOAD(Date1, Name1, Descripcion1, Hours1, Rates1, Totals1)
df1

,Date,Names,Description,Hours,Rates,Totals
0,5/30/2017,ARS,"Draft, review and revise email to client",0.20,325.0,65.0
1,5/30/2017,ARS,Phone call to client,0.40,325.0,130.0
2,5/31/2017,RE,Prepare documents for Donna Howton's Deposition,0.20,175.0,35.0
3,6/1/2017,RE,"Prepare and serve Notice in Lieu of and Notice regarding Financial Information, proof(s) of service",0.40,175.0,70.0
4,6/1/2017,ARS,"Draft, review and revise email to client",0.10,325.0,32.5
5,6/1/2017,ARS,Review email from client,0.20,325.0,65.0
6,6/1/2017,ARS,"Travel to and from and attend deposition of Donna Howton, client's wife",3.00,325.0,975.0
7,6/2/2017,EJ,"Draft, prepare, and serve Plaintiff's Objections to Notice of Deposition of Plaintiff's experts (6-7 -17)",0.30,175.0,52.5
8,6/2/2017,JRU,Prepare for dealer deposition at FCA,2.90,250.0,725.0
9,6/4/2017,JRU,Prepare for dealer deposition at FCA,0.80,250.0,200.0


# df2

In [96]:
rows_per_row = [2,2,1,2,2,2,2,1,2,2,1,2,2,2,2,2]
Date2, Name2 = Date_Name(df[2].loc[:, 0])
Descripcion2, Hours2, Rates2, Totals2 = Descripcion_Hours_Rates_Totals(df[2].loc[:, 1], rows_per_row)

In [97]:
df2 = LOAD(Date2, Name2, Descripcion2, Hours2, Rates2, Totals2)
df2

,Date,Names,Description,Hours,Rates,Totals
0,6/6/2017,JRU,Review and analyze file regarding further motion to compel deposition testimony of witnesses absent from scheduled deposition on 6/5/17,0.6,250.0,150.0
1,6/6/2017,JRU,Read and review Plaintiff's deposition transcript to prepare for drafting the TRC Report,1.5,250.0,375.0
2,6/6/2017,RE,"Draft, review and revise Joint Exhibit List",1.0,175.0,175.0
3,6/6/2017,ARS,Review email from opposing counsel Shepardson regarding expert witness availability,0.1,325.0,32.5
4,6/6/2017,ARS,"Draft, review and revise email to opposing counsel Shepardson regarding expert witness availability",0.1,325.0,32.5
5,6/6/2017,RE,"Draft, review and revise Joint Witness List, Joint Statement of the Case, Joint Jury Instructions, and Joint Special Verdict Form",0.6,175.0,105.0
6,6/6/2017,ARS,Review email from opposing counsel Tudzin regarding notice to appear for Mrs. Howton,0.1,325.0,32.5
7,6/6/2017,RE,"Draft, review and revise Joint Trial Readiness Conference Report",0.6,175.0,105.0
8,6/8/2017,ARS,"Draft, review and revise email to opposing counsel regarding exhibits for experts",0.1,325.0,32.5
9,6/8/2017,ARS,"Draft, review and revise email to opposing counsel regarding draft trial readiness conference",0.1,325.0,32.5


# df3

In [82]:
aux3 = df[3]

In [83]:
rows_per_row = [1,3,1,2,1,2,2,1,1,2,1,1,2,3,1]
Date3, Name3, Descripcion3 = Date_Name_Descripcion(aux3.loc[:, 0], rows_per_row)
Hours3, Rates3, Totals3 = Hours_Rates_Totals(aux3.loc[:, 1])

In [86]:
df3 = LOAD(Date3, Name3, Descripcion3, Hours3, Rates3, Totals3)
df3

,Date,Names,Description,Hours,Rates,Totals
0,6/9/2017,RMW,"Prepare for, travel to and from and attend TRC",4.00,500.0,2000.0
1,6/9/2017,RE,"Draft, review and revise email to Atkinson Baker regarding deposition; Draft, review and revise amended Notice of Deposition of Expert Schmidt and proof(s) of service; Serve same",0.60,175.0,105.0
2,6/9/2017,RE,Prepare documents for the deposition of FCA Expert Schmidt,0.40,175.0,70.0
3,6/12/2017,ARS,Travel to and from and attend deposition of FCA's expert Richard Schmidt in Irvine,6.50,325.0,2112.5
4,6/13/2017,ARS,Review email from client,0.10,325.0,32.5
5,6/13/2017,ARS,"Draft, review and revise email to client",0.20,325.0,65.0
6,7/12/2017,RE,"Draft, review and revise deposition subpoena for Jay Weaver; Draft, review and revise attachment 3",0.20,175.0,35.0
7,7/14/2017,JRU,"Phone call with Court clerk from Presiding, Shane, regarding trial readiness information; Compile information responsive to his questions; Follow-up voicemail to communicate information requested",0.30,250.0,75.0
8,7/27/2017,RMW,Prepare subpoena for Jay Weaver,0.30,500.0,150.0


# df4

In [66]:
rows_per_row = [2,2,1,2,2,2,3,3,4,3,1,3,4]
Date4, Name4, Descripcion4, Hours4, Rates4, Totals4 = All(df[4], rows_per_row)

In [69]:
df4 = pd.concat([Date4.rename('Date'), Name4.rename('Names'), Descripcion4.rename('Description'),
                     Hours4.rename('Hours'), Rates4.rename('Rates'), Totals4.rename('Totals')], axis=1)
df4 = df4.dropna().reset_index(drop=True)
df4

,Date,Names,Description,Hours,Rates,Totals
0,8/9/2017,ARS,Review email from opposing counsel Tudzin regarding date of Rancho dealership depositions,0.1,325.0,32.5
1,8/9/2017,RE,"Draft, review and revise Plaintiffs Third Amended Notice of Deposition of Rancho Chrysler dealership personnel; Draft, review and revise proof(s) of service; Serve same; Schedule court reporter",0.6,175.0,105.0
2,8/22/2017,JRU,Prepare for dealership deposition,1.0,250.0,250.0
3,9/8/2017,RE,"Draft, review and revise Notice in Lieu of; Draft, review and revise Letter and Notice regarding Financial Information; Draft, review and revise proof(s) of service; Draft, review and revise Nationwide regarding service; Serve same",0.3,175.0,52.5


# df5

In [92]:
df[5] = df[5].drop(5, axis=0)

In [93]:
df[5]

,0,1,2
6,9/12/2017 JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 2",0.50 125.00
7,NaN,NaN,250.00/hr
8,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 3",0.50 125.00
9,NaN,NaN,250.00/hr
10,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 4",0.50 125.00
11,NaN,NaN,250.00/hr
12,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 1",0.50 125.00
13,NaN,NaN,250.00/hr
14,9/13/2017 JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 7",0.50 125.00
15,NaN,NaN,250.00/hr


In [94]:
rows_per_row = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
Date5, Name5 = Date_Name(df[5].loc[:, 0])
Descripcion5 = Descripcion_func(df[5].loc[:, 1], rows_per_row)
Hours5, Rates5, Totals5 = Hours_Rates_Totals(df[5].loc[:, 2])

In [95]:
df5 = LOAD(Date5, Name5, Descripcion5, Hours5, Rates5, Totals5)
df5

,Date,Names,Description,Hours,Rates,Totals
0,9/12/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 2",0.50,250.0,125.0
1,9/12/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 3",0.50,250.0,125.0
2,9/12/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 4",0.50,250.0,125.0
3,9/12/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 1",0.50,250.0,125.0
4,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 7",0.50,250.0,125.0
5,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 8",0.50,250.0,125.0
6,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 9",0.50,250.0,125.0
7,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 10",0.50,250.0,125.0
8,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 11",0.50,250.0,125.0
9,9/13/2017,JRU,"Draft, review and revise Plaintiffs Motion in Limine No. 6",0.50,250.0,125.0


# df6

In [62]:
rows_per_row = [1,1,3,1,1,1,1,1,1,1]
Date6, Name6, _ = Date_Name_Descripcion(df[6].loc[:, 0], rows_per_row)
aux_series = Remove_Date_Name(df[6].loc[:, 0])
Descripcion6, Hours6, Rates6, Totals6 = Descripcion_Hours_Rates_Totals(aux_series, rows_per_row)

In [63]:
Descripcion6

0    Prepare timeline chart                                                                                                                                         
1    Draft, review and revise joint witness list                                                                                                                    
2    Prepare and serve Civil Subpoenas and On Call Agreements for  dealership personnel; Draft, review and revise email to Nationwide  regarding service; Serve same
3    Draft, review and revise email to client                                                                                                                       
4    Phone call with client                                                                                                                                         
5    Receive and review notice that case has settled                                                                                                                
6    Draft

In [65]:
df6 = pd.concat([Date6.rename('Date'), Name6.rename('Names'), Descripcion6.rename('Description'),
                     Hours6.rename('Hours'), Rates6.rename('Rates'), Totals6.rename('Totals')], axis=1)
df6 = df6.dropna().reset_index(drop=True)
df6

,Date,Names,Description,Hours,Rates,Totals
0,9/19/2017,JRU,Prepare timeline chart,2.5,250.0,625.0
1,9/19/2017,JRU,"Draft, review and revise joint witness list",0.7,250.0,175.0
2,9/19/2017,RE,"Prepare and serve Civil Subpoenas and On Call Agreements for dealership personnel; Draft, review and revise email to Nationwide regarding service; Serve same",0.6,175.0,105.0
3,9/20/2017,ARS,"Draft, review and revise email to client",0.1,325.0,32.5
4,9/20/2017,ARS,Phone call with client,0.2,325.0,65.0
5,9/21/2017,RMW,Receive and review notice that case has settled,0.1,500.0,50.0
6,9/21/2017,ARS,"Draft, review and revise trial brief",1.1,325.0,357.5
7,9/27/2017,RE,"Draft, review and revise email to co-counsel",0.1,175.0,17.5


In [99]:
DF = pd.concat([df0, df1, df2, df3, df4, df5, df6])

In [100]:
DF

,Date,Names,Description,Hours,Rates,Totals
0,5/4/2017,RMW,Receive and review RO Summary,0.30,500.0,150.0
1,5/4/2017,RMW,Review and analyze case files for developing trial strategy,2.00,500.0,1000.0
2,5/24/2017,EJ,"Draft, prepare, and serve Plaintiff's Amended Notice of Deposition of Rancho Chrysler (6-5-17)",0.80,175.0,140.0
3,5/25/2017,ARS,"Draft, review and revise email to Donna Howton",0.10,325.0,32.5
4,5/26/2017,RE,"Draft, review and revise notice in lieu of and notice regarding financial information",0.50,175.0,87.5
5,5/30/2017,EJ,"Draft, prepare, and serve Plaintiff's Objections to Notice of Deposition of Mrs. Howton (6-1-17)",0.20,175.0,35.0
0,5/30/2017,ARS,"Draft, review and revise email to client",0.20,325.0,65.0
1,5/30/2017,ARS,Phone call to client,0.40,325.0,130.0
2,5/31/2017,RE,Prepare documents for Donna Howton's Deposition,0.20,175.0,35.0
3,6/1/2017,RE,"Prepare and serve Notice in Lieu of and Notice regarding Financial Information, proof(s) of service",0.40,175.0,70.0


In [101]:
DF.to_excel(r"C:\Users\brian\Desktop\trabajillo\\1088878.xlsx", index=False)